In [1]:
import numpy as np
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data
import time

In [2]:
#1.Import Data(MNIST)
mnist = input_data.read_data_sets('mnist', one_hot=True)

Extracting mnist/train-images-idx3-ubyte.gz
Extracting mnist/train-labels-idx1-ubyte.gz
Extracting mnist/t10k-images-idx3-ubyte.gz
Extracting mnist/t10k-labels-idx1-ubyte.gz


In [3]:
print(mnist.train.labels.shape)
print(mnist.test.labels.shape)
print(mnist.train.images[0].shape)

(55000, 10)
(10000, 10)
(784,)


In [4]:
#Placeholder
x_data = tf.placeholder(tf.float32,[None,784], name = 'X_placeholder')
y_target = tf.placeholder(tf.float32,[None,10], name ='Y_placeholder')

In [5]:
#Parameter
in_num = 784
classes= 10
hidden_1 = 256
hidden_2 = 256

#initialize w / b
w = {'h1':tf.Variable(tf.random_normal([in_num, hidden_1]), name='W1'),
    'h2':tf.Variable(tf.random_normal([hidden_1,hidden_2]), name='W2'),
    'out':tf.Variable(tf.random_normal([hidden_2,classes]),name='W')}

b = {
    'b1':tf.Variable(tf.random_normal([hidden_1]), name='b1'),
    'b2':tf.Variable(tf.random_normal([hidden_2]),name='b2'),
    'out':tf.Variable(tf.random_normal([classes]),name='b')
}

In [6]:
def multilayer_perceptron(x, w, b):
    # Hidden1，ReLu
    layer_1 = tf.add(tf.matmul(x, w['h1']), b['b1'], name='fc_1')
    layer_1 = tf.nn.relu(layer_1, name='relu_1')
    # Hidden2，ReLu
    layer_2 = tf.add(tf.matmul(layer_1, w['h2']), b['b2'], name='fc_2')
    layer_2 = tf.nn.relu(layer_2, name='relu_2')
    # 输出层
    out_layer = tf.add(tf.matmul(layer_2, w['out']), b['out'], name='fc_3')
    return out_layer

In [7]:
pred = multilayer_perceptron(x_data, w, b)

In [8]:
learning_rate = 0.01
loss_all = tf.nn.softmax_cross_entropy_with_logits(logits=pred, labels=y_target, name='cross_entropy_loss')
loss = tf.reduce_mean(loss_all, name='avg_loss')
optimizer = tf.train.AdamOptimizer(learning_rate).minimize(loss)

In [9]:
# training times
training_epochs = 30
#一批数据大小
batch_size = 128
#信息展示的频度
display_step = 5

init = tf.global_variables_initializer()

with tf.Session() as sess:
    sess.run(init)
    writer = tf.summary.FileWriter('./graphs/MLP_DNN', sess.graph)

    # 训练
    for epoch in range(training_epochs):
        avg_loss = 0.
        total_batch = int(mnist.train.num_examples/batch_size)
        # 遍历所有的batches
        for i in range(total_batch):
            batch_x, batch_y = mnist.train.next_batch(batch_size)
            # 使用optimizer进行优化
            _, l = sess.run([optimizer, loss], feed_dict={x_data: batch_x, y_target: batch_y})
            # 求平均的损失
            avg_loss += l / total_batch
        # 每一步都展示信息
        if epoch % display_step == 0:
            print("Epoch:", '%04d' % (epoch+1), "cost=", "{:.9f}".format(avg_loss))
    print("Optimization Finished!")

    # 在测试集上评估
    correct_prediction = tf.equal(tf.argmax(pred, 1), tf.argmax(y_target, 1))
    # 计算准确率
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, "float"))
    print("Accuracy:", accuracy.eval({x_data: mnist.test.images, y_target: mnist.test.labels}))
    writer.close()

Epoch: 0001 cost= 62.957781597
Epoch: 0006 cost= 2.180951942
Epoch: 0011 cost= 1.441058417
Epoch: 0016 cost= 0.958620226
Epoch: 0021 cost= 0.734329941
Epoch: 0026 cost= 0.661575954
Optimization Finished!
Accuracy: 0.9671
